In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv("/Users/deepika/Documents/HBDDs/Human-Behavior-Drift-Detection-System/data/trajectory_data/trajectory_data_1.csv")

df = df.sort_values(["person_id","time"])

df["dx"] = df.groupby("person_id")["x"].diff()
df["dy"] = df.groupby("person_id")["y"].diff()

df["speed"] = np.sqrt(df["dx"]**2 + df["dy"]**2)


In [4]:
df["direction"] = np.arctan2(df["dy"], df["dx"])


In [5]:
df["acceleration"] = df.groupby("person_id")["speed"].diff()


In [6]:
density = df.groupby("time")["person_id"].nunique().reset_index()
density.columns = ["time","density"]
df = df.merge(density, on="time")


In [7]:
def entropy(series, bins=8):
    hist, _ = np.histogram(series.dropna(), bins=bins)
    p = hist / hist.sum()
    p = p[p>0]
    return -np.sum(p*np.log(p))

entropy_df = df.groupby("time")["direction"].apply(entropy).reset_index()
entropy_df.columns = ["time","direction_entropy"]
df = df.merge(entropy_df, on="time")


/var/folders/9f/qjdzglkx7gj3cg8lrg5c4vk80000gn/T/ipykernel_9435/3551710162.py:3: RuntimeWarning: invalid value encountered in divide
  p = hist / hist.sum()


In [8]:
from scipy.spatial import KDTree

def avg_nearest_distance(group):
    pts = group[["x","y"]].values
    if len(pts) < 2:
        return 0
    tree = KDTree(pts)
    dists, _ = tree.query(pts, k=2)
    return np.mean(dists[:,1])

cluster_df = df.groupby("time").apply(avg_nearest_distance).reset_index()
cluster_df.columns = ["time","avg_neighbor_dist"]
df = df.merge(cluster_df, on="time")


In [9]:
features = df[["time","speed","direction","density","direction_entropy","avg_neighbor_dist"]]
features.to_csv("behavior_features.csv", index=False)
